# Продуктовая аналитика: Метрики продукта

Обсудим, что такое метрики и зачем они нужны, что творится на рынке в отношении этих метрик и какие существуют особенности расчёта метрик в различных инструментах. Также поговорим про физический смысл и интерпретацию метрик и потренируемся считать метрики по сырым данным.

### Проект: Доставка продуктов

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события
- event - событие

    - app_install – установка приложения
    - app_start – открыть приложения
    - register – зарегистрироваться 
    - search – перейти на страницу поиска товаров (каталог)
    - open_item – открыть товар
    - choose_item – отправить товар в корзину
    - tap_basket – перейти в корзину
    - purchase – подтверждение покупки
    
- gender – пол пользователя
- os_name – платформа пользователя
- city – город пользователя
- device_id – идентификатор устройства пользователя
- urm_source – канал, с которого пришел пользователь

    - yandex-direct – Яндекс директ
    - google_ads – реклама в Google
    - vk_ads – реклама в ВК
    - instagram_ads – реклама в instagram
    - facebook_ads – реклама в facebook
    - referal – акция «приведи друга»
    - Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
    
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)


Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [81]:
kc = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-eremina/KC_case_data.csv')

In [82]:
kc.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


Какие две метрики, по сравнению с остальными, поможет ответить на вопрос «Насколько хорошо маркетологи справляются со своей задачей привлечения новых пользователей»?


- DAU
- **CAC**
- Количество сессий
- **Количество инсталлов**
- Retention 7-го дня


Какая не денежная метрика в динамике лучше всего отражает нравится ли продукт пользователям?

- Выручка
- CR из установки в открытие приложения
- CR из открытия приложения в покупку
- MAU
- **CR в повторную покупку в течение месяца**

MAU февраля:

In [83]:
kc['date'] = pd.to_datetime(kc.date)

In [84]:
kc.query('date.dt.month == 2').device_id.nunique()

75032

Количество установок в январе:

In [85]:
kc.query("date.dt.month == 1 and event == 'app_install'").device_id.count()

80297

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая

In [86]:
first_reg = kc.query('event == "register"').sort_values('date').drop_duplicates('device_id')
kc['first_reg'] = kc['device_id'].map(first_reg.set_index('device_id')['date'])

In [87]:
already_reg = kc.query('first_reg < date')

In [88]:
already_reg.groupby('event')['device_id'].nunique()

event
app_start      40991
choose_item    37926
purchase       24880
search         40482
tap_basket     34517
Name: device_id, dtype: int64

In [89]:
# конверсия из регистрации в поиск
40482/40991

0.9875826400917275

In [90]:
# конверсия из поиска в добавление товара
37926/40991

0.9252274889609914

In [91]:
# конверсия из добавления товара в переход в корзину
34517/37926

0.9101144333702473

In [92]:
# конверсия из перехода в корзину в покупку
24880/34517

0.7208042413883015

Переход в корзину - минимум

Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [97]:
df = kc.query('event == "purchase"').groupby('utm_source', as_index = False) \
       .agg({'device_id' : 'nunique'}).rename(columns={'device_id' : 'count_purchase'})
df

,utm_source,count_purchase
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [98]:
df1 = kc.query('event == "app_start"').groupby('utm_source', as_index = False) \
  .agg({'device_id' : 'nunique'}).rename(columns={'device_id' : 'count_start'})
df1 

,utm_source,count_start
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [99]:
df = df.merge(df1, on = 'utm_source')
df['CR'] = df.count_purchase / df.count_start
df = df.sort_values('CR', ascending=False)
df

,utm_source,count_purchase,count_start,CR
4,referal,6362,15924,0.399523
5,vk_ads,12364,34079,0.362804
1,facebook_ads,9017,25957,0.347382
3,instagram_ads,10762,31045,0.346658
0,-,16598,52268,0.317556
2,google_ads,11339,38092,0.297674
6,yandex-direct,12028,40707,0.295477


У Яндкса самая низкая конверсия в покупку

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».



In [100]:
check = kc.query('event == "purchase"').groupby(['utm_source', 'device_id'],as_index=False).agg({'purchase_sum' : 'first'})
check

,utm_source,device_id,purchase_sum
0,-,4046,429.5
1,-,4120,582.5
2,-,4202,552.0
3,-,4214,231.5
4,-,4234,583.5
...,...,...,...
78465,yandex-direct,35245401,202.5
78466,yandex-direct,35258588,157.0
78467,yandex-direct,35266881,80.0
78468,yandex-direct,35277913,81.0


In [101]:
check.groupby('utm_source',as_index=False).agg({'purchase_sum' : 'median'}).sort_values('purchase_sum', ascending = False)

,utm_source,purchase_sum
0,-,398.5
4,referal,395.0
3,instagram_ads,393.5
5,vk_ads,393.0
6,yandex-direct,392.5
1,facebook_ads,390.5
2,google_ads,390.0


Медианный первый чек выше у пользователей с реферальной программы

Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм».

### ROMI = (вал.прибыль - расходы на рекламу)/расходы на рекламу

In [102]:
kc.query('event == "purchase"').groupby('utm_source', as_index=False).agg({'purchase_sum' : 'sum'})

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [103]:
# facebook_ads
(12249901.0 - 8590498) / 8590498

0.4259826380263403

In [104]:
# google_ads
(12868276.0 - 10534878) / 10534878 

0.22149264566708793

In [105]:
# instagram_ads
(14546969.0 - 8561626) / 8561626

0.6990895187432854

In [106]:
# vk_ads
(16389652.5 - 9553531) / 9553531

0.7155596710786828

ВК имеет самый высокий ROMI